<a href="https://colab.research.google.com/github/YatinGupta777/Whatsapp-Chat-Analyzer/blob/master/WhatsApp_Chat_Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import re
import pandas as pd
from matplotlib import pyplot as plt
from pandas import ExcelWriter
from google.colab import files
import numpy as np

In [0]:

def startsWithDate(s):

    #MM/DD/YY
    pattern = '^(([0-9])|((1)[0-2]))(\/)([0-9])|([0-2][0-9]|(3)[0-1])(\/)(\d{2}|\d{4})'
    result = re.match(pattern, s)
    if result:
        return True
    return False


def startsWithAuthor(s):
    patterns = [
        '([\w]+):',                        # First Name
        '([\w]+[\s]+[\w]+):',              # First Name + Last Name
        '([\w]+[\s]+[\w]+[\s]+[\w]+):',    # First Name + Middle Name + Last Name
        '([+]\d{2} \d{5} \d{5}):',         # Mobile Number (India)
        '([+]\d{2} \d{3} \d{3} \d{4}):',   # Mobile Number (US)
        '([+]\d{2} \d{4} \d{7})'           # Mobile Number (Europe)
    ]
    pattern = '^' + '|'.join(patterns)
    result = re.match(pattern, s)
    if result:
        return True
    return False

def getDataPoint(line):
    # line = 18/06/17, 22:47 - Loki: Why do you have 2 numbers, Banner?
    
    splitLine = line.split(' - ') # splitLine = ['18/06/17, 22:47', 'Loki: Why do you have 2 numbers, Banner?']
    
    dateTime = splitLine[0] # dateTime = '18/06/17, 22:47'
    
    date, time = dateTime.split(', ') # date = '18/06/17'; time = '22:47'
    
    message = ' '.join(splitLine[1:]) # message = 'Loki: Why do you have 2 numbers, Banner?'
    
    if startsWithAuthor(message): # True
        splitMessage = message.split(': ') # splitMessage = ['Loki', 'Why do you have 2 numbers, Banner?']
        author = splitMessage[0] # author = 'Loki'
        message = ' '.join(splitMessage[1:]) # message = 'Why do you have 2 numbers, Banner?'
    else:
        author = None
    return date, time, author, message

parsedData = [] # List to keep track of data so it can be used by a Pandas dataframe
conversationPath = '/content/drive/My Drive/Colab Notebooks/WhatsApp-Chat-Analyzer/Chat.txt' 
with open(conversationPath, encoding="utf-8") as fp:
    fp.readline() # Skipping first line of the file (usually contains information about end-to-end encryption)
        
    messageBuffer = [] # Buffer to capture intermediate output for multi-line messages
    date, time, author = None, None, None # Intermediate variables to keep track of the current message being processed
    
    while True:
        line = fp.readline() 
        if not line: # Stop reading further if end of file has been reached
            break
        line = line.strip()
       # Guarding against erroneous leading and trailing whitespaces
        if startsWithDate(line): # If a line starts with a Date Time pattern, then this indicates the beginning of a new message
            if len(messageBuffer) > 0: # Check if the message buffer contains characters from previous iterations
                parsedData.append([date, author])#' '.join(messageBuffer)]) # Save the tokens from the previous message in parsedData
            messageBuffer.clear() # Clear the message buffer so that it can be used for the next message
            date, time, author, message = getDataPoint(line) # Identify and extract tokens from the line
            messageBuffer.append(message) # Append message to buffer
        else:
            messageBuffer.append(line) # If a line doesn't start with a Date Time pattern, then it is part of a multi-line message. So, just append to buffer
#pd.set_option('max_rows', 99999)
#pd.set_option('max_colwidth', 400)
#pd.describe_option('max_colwidth')

df = pd.DataFrame(parsedData, columns=['Date', 'Author'])
df_by_date = df.groupby(['Date'],sort=False).size().reset_index()
df_by_date_author = df.groupby(['Author','Date'],sort=False).size().to_frame('Messages').reset_index()
#df_new.sort_values()
#print(df_new.head)
#df.describe()

#df_by_date_author.describe()

#df['Date'].value_counts().plot.barh() # Top 10 Times of the day at which the most number of messages were sent
#plt.xlabel('Number of messages')
#plt.ylabel('Time')



In [5]:
#df_by_date_author.columns['Date','Author','Messages']
#print(type(df_by_date_author))
df_by_date_author.head()

,Author,Date,Messages
0,Pranav,6/10/18,3
1,Daljit,6/10/18,1
2,Aayush,6/10/18,1
3,Pranav,6/11/18,8
4,Daljit,6/11/18,7


In [44]:
authors = df['Author'].unique()
dates = df['Date'].unique()

authors = [x for x in authors if x is not None]

month_year_dates = []
for i in dates:
  x = i.partition("/")
  y = x[2].partition("/")
  month_year = x[0] +"/" + y[2]
  if month_year not in month_year_dates: 
    month_year_dates.append(month_year)

#month_year_dates = set(month_year_dates)
authors

['Pranav',
 'Daljit',
 'Aayush',
 'Yash',
 'Shivam',
 'Yatin Gupta',
 'Himanshu',
 'Amit',
 'Tanya',
 'Himanshi',
 'Shivam Rastogi']

In [45]:
df_structure = pd.DataFrame(index=authors, columns=month_year_dates)
df_structure.fillna(0, inplace=True)
df_structure

,6/18,7/18,8/18,9/18,10/18,11/18,12/18,1/19,2/19,3/19,4/19,5/19,6/19,7/19,8/19,9/19,10/19,11/19,12/19,1/20,2/20,3/20
Pranav,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Daljit,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Aayush,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Yash,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Shivam,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Yatin Gupta,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Himanshu,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Amit,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Tanya,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Himanshi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [46]:
for index, row in df_by_date_author.iterrows():
  #print(row['Date'],row['Author'],row['Messages'])
  x = row['Date'].partition("/")
  y = x[2].partition("/")
  month_year = x[0] +"/" + y[2]
  temp = df_structure.at[row['Author'],month_year]
  temp = temp + row['Messages']
  df_structure.set_value(row['Author'],month_year,temp)
 
df_structure
  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


,6/18,7/18,8/18,9/18,10/18,11/18,12/18,1/19,2/19,3/19,4/19,5/19,6/19,7/19,8/19,9/19,10/19,11/19,12/19,1/20,2/20,3/20
Pranav,278,192,23,59,76,169,170,38,99,298,309,706,659,236,111,178,226,73,61,78,116,268
Daljit,613,672,423,237,244,544,632,298,173,650,791,3679,651,392,156,80,251,106,88,169,130,381
Aayush,190,174,15,49,77,247,230,111,159,551,338,1005,364,113,59,43,171,53,66,40,78,228
Yash,333,434,182,159,285,158,371,179,176,633,568,1347,383,133,51,132,80,47,0,0,4,64
Shivam,88,86,25,17,1,24,103,41,93,78,243,364,112,98,22,49,83,22,98,88,143,380
Yatin Gupta,231,244,79,85,125,175,123,145,95,223,274,699,215,62,15,106,142,40,22,27,63,179
Himanshu,167,194,151,123,162,271,405,182,143,346,405,626,353,171,108,208,365,61,41,53,94,297
Amit,116,57,71,46,56,157,352,147,82,263,59,273,157,158,17,50,65,15,12,26,50,122
Tanya,0,0,0,0,0,0,0,0,0,0,0,877,0,0,0,0,0,0,0,0,0,0
Himanshi,0,0,0,0,0,0,0,0,0,0,0,138,0,0,0,0,0,0,0,0,0,0


In [47]:
df_structure

,6/18,7/18,8/18,9/18,10/18,11/18,12/18,1/19,2/19,3/19,4/19,5/19,6/19,7/19,8/19,9/19,10/19,11/19,12/19,1/20,2/20,3/20
Pranav,278,192,23,59,76,169,170,38,99,298,309,706,659,236,111,178,226,73,61,78,116,268
Daljit,613,672,423,237,244,544,632,298,173,650,791,3679,651,392,156,80,251,106,88,169,130,381
Aayush,190,174,15,49,77,247,230,111,159,551,338,1005,364,113,59,43,171,53,66,40,78,228
Yash,333,434,182,159,285,158,371,179,176,633,568,1347,383,133,51,132,80,47,0,0,4,64
Shivam,88,86,25,17,1,24,103,41,93,78,243,364,112,98,22,49,83,22,98,88,143,380
Yatin Gupta,231,244,79,85,125,175,123,145,95,223,274,699,215,62,15,106,142,40,22,27,63,179
Himanshu,167,194,151,123,162,271,405,182,143,346,405,626,353,171,108,208,365,61,41,53,94,297
Amit,116,57,71,46,56,157,352,147,82,263,59,273,157,158,17,50,65,15,12,26,50,122
Tanya,0,0,0,0,0,0,0,0,0,0,0,877,0,0,0,0,0,0,0,0,0,0
Himanshi,0,0,0,0,0,0,0,0,0,0,0,138,0,0,0,0,0,0,0,0,0,0


In [0]:
writer = pd.ExcelWriter('monthwise_messages.xlsx')
df_structure.to_excel(writer)
writer.save()
files.download('monthwise_messages.xlsx')
